Extracting matrix from mne object, creating a log and calling function to find the pulses

In [ ]:
def main( ): #main arguments: raw, logs, max = none, min = none
    raw = mne.io.read_raw_brainvision('sub100_rt_TEP.vhdr')
    mat = raw.get_data()
    output_log = {}
    num_of_artifacts = {}
    logs = get_interpolation(raw)


In [ ]:
def get_interpolation(raw):
    logs = {}
    logs['Errors'] = []
    logs['General Info'] = {}
    mat = raw.get_data()  # create 65 row matrix with ~7200000. Row = brain area, Column = 0.1 ms
    diffs = np.diff(mat)
    peaks = get_peak(mat)

Determine abnormal values in differences matrix

In [ ]:
def get_peak(mat):
    max_mat = np.amax(mat, axis=1)
    min_mat = np.amin(mat, axis=1)
    peaks = (max_mat + np.abs(min_mat)) / 7
    return peaks


Iterating over matrix's rows, for each row, apply all of the followings: 

In [ ]:
for i in range(len(mat)):  # for each region
        peak = peaks[i]  # get_peak(max_mat[i], min_mat[i])  #determine required amplitude
        logs['Region ' + str(i + 1)] = {'Amplitude': peak}
        indices = find_abnormalities(diffs[i], peak, i)  # find abnormal indices
        pulses_lst = group_indices(indices, 6)  # group abnormal indices by adjacency
        sort_pulses_for_region(pulses_lst)  # sort indices
        final_array = check_shape(mat[i], pulses_lst, peak, i)  # check pattern of pulses contains min&max values
        logs['Region ' + str(i + 1)]['Number of pulses'] = len(final_array)
        logs['Region ' + str(i + 1)]['Indices'] = final_array
    lst = find_range(logs, len(mat), logs['Region 1']['Number of pulses'])  # determine final locations of pulses
    logs['General Info']["Indices to interpolate"] = lst
    return logs

Using the value from get_peaks, find abnormal indices

In [ ]:
def find_abnormalities(diffs, peak, j):
    low_inds = np.where(diffs > peak)
    high_inds = np.where(diffs < (-peak))
    return (high_inds, low_inds)


Because each pulse contains several abnormal indices, we will group all indices that are at most 0.6ms apart into one pulse 

In [ ]:
def group_indices(indices, freq):
    high_inds = indices[0][0]
    low_inds = indices[1][0]
    pulses_lst = [[]]  # initiate grouped abnormal indices
    group_first_by_low(low_inds, pulses_lst, freq)
    add_high_to_groups(high_inds, pulses_lst, freq)
    return pulses_lst

Example of grouping abnormal low values

In [ ]:
def group_first_by_low(low_inds, pulses_lst, freq):  # creates a nested list for each suspected pulse
    cnt = 0
    for i in range(len(low_inds)):  # Loop to group values to pulses
        pulses_lst[cnt].append(low_inds[i])
        try:  # avoid error incase we reached end of list indices
            if abs(low_inds[i] - low_inds[i + 1]) > freq:  # If next index is more than 0.6 ms apart - NOT the same pulse
                cnt += 1
                pulses_lst.append([])  # Add another row for the next pulse of this area
        except IndexError:
            cnt += 1
            pass
    return

Sort all indices within each pulse after high and low grouping

In [ ]:
def sort_pulses_for_region(pulses_lst):
    for i in range(len(pulses_lst)):
        pulses_lst[i] = np.sort(pulses_lst[i])

For each pulse we make sure it contains at least one abnormal high value and at least one abnormal low value - to match the required pattern of TMS pulse

In [ ]:
def check_shape(row, pulses_lst, peak, j):
    final_array = []
    for i in range(len(pulses_lst)):
        if len(pulses_lst[i]) == 0:
            continue
        values = row[pulses_lst[i]]
        inds = [k for k in range(np.amin(pulses_lst[i] - 10), np.amax(pulses_lst[i] + 10))]
        med = np.median(row[inds])
        if np.amin(values) < med - peak / 3 and np.amax(values) > med + peak / 3:
            final_array.append(pulses_lst[i].tolist())
    return final_array

For each pulse, we find the average pulse start point and the average pulse endpoint and create a 2ms segment that contains the pulse

In [ ]:
def find_range(logs, rows, pulses):  # take average of all indices and create a 2 ms section
    result = []
    for i in range(pulses):
        min_sum = 0
        max_sum = 0
        cnt = 0
        for j in range(rows):
            try:
                max_sum += logs['Region ' + str(j + 1)]['Indices'][i][-1]
                min_sum += logs['Region ' + str(j + 1)]['Indices'][i][0]
            except IndexError:
                cnt += 1
                logs['Errors'].append("Error in row " + str(j) + " In pulse " + str(i))
        max_avg = max_sum // (rows - cnt)
        min_avg = min_sum // (rows - cnt)
        diff = max_avg - min_avg
        extension = (20 - diff) // 2
        result.append([min_avg - extension, max_avg + extension])
    return result

Returning to main flow, we create a new matrix, then iterate over the rows

In [ ]:
interpolated_mat = np.copy(mat)
    for channel in range(len(mat)): #iterating over the channels
        key = "Region " + str(channel+1)
        compare_to_events(raw , logs['General Info']["Indices to interpolate"])
        xaxis , yaxis , inter_xaxis = interpolation_axises(mat[channel] , logs['General Info']["Indices to interpolate"] )
        inter_yaxis = interpolation(xaxis, yaxis, inter_xaxis ,mat[channel])
        for x in range(len(inter_xaxis)):
            interpolated_mat[channel][inter_xaxis[x]] = inter_yaxis[x]

We create an x-axis without the contaminated indexes and insert the original y-axis values of the uncontaminated indices

In [ ]:
def interpolation_axises(row , indices):
    xaxis = [ i for i in range(indices[0][0])]
    new_xaxis = []
    for index in range(len(indices)):
        try : 
            new_xaxis += [i for i in range(indices[index][0] , indices[index][1] + 1)]
            xaxis += [i for i in range(indices[index][1] , indices[index + 1][0])]
        except IndexError:
            xaxis += [i for i in range(indices[index][1] , len(row))]  
    yaxis = [row[i] for i in xaxis] 
    print("num of xaxises to interpolate , " + str(len(new_xaxis)))
    return xaxis , yaxis , new_xaxis

Using the uncontaminated x,y values, we create a cubic interpolation function based on our data, create y values for our contaminated indices and insert the values in the new matrix 

In [ ]:
def interpolation(xaxis , yaxis, inter_xaxis, row):
    # returns a new matrix with the values created by the interpolation
    inter_func = interpolate.interp1d(xaxis, yaxis, kind = 'cubic')
    interpolated_row = inter_func(inter_xaxis)
    #check_interpolation(inter_func, xaxis, yaxis, row) 
    return interpolated_row

Print the log that contains the desired information

Create a new mne object with the interpolated matrix values and return it

In [ ]:
info = raw.info  # Preserve the original info structure
raw_new = mne.io.RawArray(interpolated_mat, info)
return raw_new